In [13]:
import pandas as pd

data = pd.read_csv('train.csv')

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [27]:
print(train_data.columns)
print(val_data.columns)

Index(['qid', 'question_text', 'target', 'cleaned_text'], dtype='object')
Index(['qid', 'question_text', 'target', 'cleaned_text'], dtype='object')


In [28]:
import pandas as pd

# Load your dataset
data = pd.read_csv('train.csv')

# Clean the data
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stop words
    return text

data['cleaned_text'] = data['question_text'].apply(clean_text)

# Create train/validation splits
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
import numpy as np

def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_file = 'glove.6B.50d.txt'
embeddings_index = load_glove_embeddings(glove_file)

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['cleaned_text'])

train_sequences = tokenizer.texts_to_sequences(train_data['cleaned_text'])
val_sequences = tokenizer.texts_to_sequences(val_data['cleaned_text'])

max_sequence_length = 100
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_padded = pad_sequences(val_sequences, maxlen=max_sequence_length)

In [31]:
word_index = tokenizer.word_index
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_sequence_length,
                    trainable=False))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [33]:
model.fit(train_padded, train_data['target'],
          validation_data=(val_padded, val_data['target']),
          epochs=5,
          batch_size=32)

Epoch 1/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4095s 125ms/step - accuracy: 0.9377 - loss: 0.2353 - val_accuracy: 0.9393 - val_loss: 0.2300
Epoch 2/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 3949s 121ms/step - accuracy: 0.9379 - loss: 0.2331 - val_accuracy: 0.9393 - val_loss: 0.2289
Epoch 3/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4132s 127ms/step - accuracy: 0.9380 - loss: 0.2327 - val_accuracy: 0.9393 - val_loss: 0.2293
Epoch 4/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4443s 136ms/step - accuracy: 0.9374 - loss: 0.2341 - val_accuracy: 0.9393 - val_loss: 0.2293
Epoch 5/5
32654/32654 ━━━━━━━━━━━━━━━━━━━━ 4422s 135ms/step - accuracy: 0.9380 - loss: 0.2325 - val_accuracy: 0.9393 - val_loss: 0.2291


In [35]:
model.save('spam_filter_model.keras')

In [36]:
from tensorflow.keras.models import load_model

model = load_model('spam_filter_model.keras')

C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [41]:
model.save('my_model.keras')  # For the native Keras format

In [42]:
import os

# Replace 'my_model.keras' or 'my_model.h5' with your file name
file_name = 'my_model.keras'
if os.path.exists(file_name):
    print(f"File '{file_name}' saved successfully.")
else:
    print(f"File '{file_name}' was not saved.")

File 'my_model.keras' saved successfully.


In [43]:
import os
print(os.getcwd())  # Prints the directory where the script is running

C:\Users\DELL
